In [2]:
# In case you do not have following packages installed, uncomment instalisation.

import pandas as pd
import numpy as np
import os
import glob
import functools
from pathlib import Path
import matplotlib.pyplot as plt

#!pip install dask[complete];
# you need to run these in case dask gives you error, it might need update.
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade
#import dask.dataframe as dd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance       

from sklearn.feature_selection import SelectFromModel
from sklearn.utils import class_weight

#!pip3 install xgboost
from xgboost import XGBClassifier

#!pip install conorm
#import conorm # for tmm normalisation

#!pip3 install pydeseq2 or pip install pydeseq2
#from pydeseq2.dds import DeseqDataSet
#from pydeseq2.ds import DeseqStats
#from pydeseq2.utils import load_example_data



#to install R :
#conda install -c r r-irkernel

#to install a library from R
#!pip install library edgeR
# pip install rpy2

### Data for Imaging of Parkinson ###

In [2]:
#path2 = Path("/Users/zainabnazari/data") # in your laptop, where the output data will be saved at the end. TAC

In [3]:
path2 = Path("/home/znazari/data") # in your laptop, where the output data will be saved at the end. TAC

DATSCAN_LIGAND = Signal CAUDATE_R	and else different part of the brain  6 different

In [4]:
imaging = pd.read_csv(path2/"DaTScan_Analysis_only_SC.txt",delimiter='\t')

In [4]:
imaging

,Diagnosis,PROTOCOL,PATNO,EVENT_ID,DATSCAN_LIGAND,DATSCAN_DATE,DATSCAN_CAUDATE_R,DATSCAN_CAUDATE_L,DATSCAN_PUTAMEN_R,DATSCAN_PUTAMEN_L,DATSCAN_PUTAMEN_R_ANT,DATSCAN_PUTAMEN_L_ANT,DATSCAN_ANALYZED
0,Healthy Control,1,3000,SC,NaN,apr-13,3.28,3.20,2.53,2.71,2.93,3.33,Yes
1,Healthy Control,1,3008,SC,NaN,set-10,2.13,1.82,1.75,1.04,2.16,1.42,Yes
2,Healthy Control,1,3009,SC,NaN,ott-10,1.73,1.93,0.95,0.74,1.30,1.45,Yes
3,Healthy Control,1,3011,SC,NaN,nov-10,1.91,1.04,1.12,0.43,1.44,0.67,Yes
4,Healthy Control,1,3013,SC,NaN,nov-10,2.11,2.26,1.15,1.58,1.56,2.25,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,Parkinson's Disease,2,162929,SC,123I-DaTscan,mar-22,1.67,2.07,0.76,0.47,1.04,1.00,Yes
1239,Parkinson's Disease,2,162994,SC,123I-DaTscan,apr-22,1.22,1.39,0.38,0.73,0.66,1.03,Yes
1240,Parkinson's Disease,2,163265,SC,123I-DaTscan,mar-22,1.60,2.32,0.58,1.35,1.05,1.75,Yes
1241,Parkinson's Disease,2,164900,SC,123I-DaTscan,dic-21,2.88,2.75,1.89,1.89,2.39,2.36,Yes


In [5]:
columns_to_remove=['PROTOCOL', 'EVENT_ID','DATSCAN_LIGAND','DATSCAN_DATE','DATSCAN_ANALYZED']
imaging_=imaging.drop(columns=columns_to_remove)

In [36]:
imaging['EVENT_ID'].unique()

array(['SC'], dtype=object)

In [11]:
imaging['Diagnosis']

0           Healthy Control
1           Healthy Control
2           Healthy Control
3           Healthy Control
4           Healthy Control
               ...         
1238    Parkinson's Disease
1239    Parkinson's Disease
1240    Parkinson's Disease
1241    Parkinson's Disease
1242    Parkinson's Disease
Name: Diagnosis, Length: 1243, dtype: object

In [28]:
imaging_['Diagnosis'].unique()

array(['Healthy Control', "Parkinson's Disease"], dtype=object)

In [6]:
# Convert labels to numeric values using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(imaging['Diagnosis'])

In [55]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [17]:
X = imaging_.drop(['PATNO', 'Diagnosis'], axis=1)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
model = LogisticRegression()

In [16]:
# Create and train the XGBoost model
#model = XGBClassifier()
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [58]:
# Predict probabilities on the test set
y_pred_prob = model.predict_proba(X_test)[:, 1]

In [59]:
# Calculate AU-ROC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"Area Under the ROC Curve (AU-ROC) score: {roc_auc}")

Area Under the ROC Curve (AU-ROC) score: 0.5316127694859039


In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu


results = []

# Iterate over each feature and perform the Mann-Whitney U test
for feature in imaging_.columns[:-1]:  # Exclude the 'group' column
    group_a_values = imaging_[imaging_['DATSCAN_CAUDATE_R'] == 'Group_A'][feature]
    group_b_values = imaging_[imaging_['group'] == 'Group_B'][feature]

    # Perform Mann-Whitney U test
    stat, p_value = mannwhitneyu(group_a_values, group_b_values, alternative='two-sided')

    # Store the results
    results.append({'Feature': feature, 'Stat': stat, 'P-value': p_value})

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)


In [62]:
#command+/v to uncomment 
# # I tried to find the best hyperparameter for this model but it didn't work well as the best score was not better than the default hyperparameter that was selested by the XGboost itself.

# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer

# # Assuming df is your DataFrame with features and labels

# # Feature engineering, preprocessing, and XGBoost in a pipeline
# pipeline = Pipeline([
#     ('preprocessor', ColumnTransformer(transformers=[
#         ('num', StandardScaler(), ['DATSCAN_CAUDATE_R', 'DATSCAN_CAUDATE_L', 'DATSCAN_PUTAMEN_R', 'DATSCAN_PUTAMEN_L', 'DATSCAN_PUTAMEN_R_ANT', 'DATSCAN_PUTAMEN_L_ANT']),   
#     ])),
#     ('classifier', XGBClassifier())
# ])

# # Define hyperparameters for grid search
# param_grid = {
#     'classifier__n_estimators': [400,500,600]
# }

# # Use grid search to find the best hyperparameters
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
# grid_search.fit(imaging_.drop(['PATNO', 'Diagnosis'], axis=1), y)

# # Display the best parameters and AU-ROC score
# print("Best Parameters:", grid_search.best_params_)
# print("Best AU-ROC Score:", grid_search.best_score_)


In [ ]:
# I have tried to standarized the data with features using a well-known module in pyhthon 
# I noticed the performance dropped from 53 to 49 Au-Roc score !

#removing noise and standardizing the features, it 
#ensures that each feature has a mean of 0 and a standard deviation of 1.
#scaler=StandardScaler()
#X_standardized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)